### Import libraries and data

In [1]:
import pandas as pd
import csv
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
vols = pd.read_csv("volumes.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [3]:
temps = pd.read_csv("temperatures.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [4]:
#headers = ["Ru", "Rb", "Rh", "Be", "Ba", "Bi", "Br", "H", "P", "Ge", "Gd", "Ga", "Pr", "Pu", "C",\
#"Pd", "Cd", "Ho", "Mg", "Mo", "Mn", "O", "S", "Eu", "Zr", "Er", "Ni",\
#"Na", "Nb", "Nd", "Fe", "B", "F", "Sr", "N", "Si", "Sn", "Sm", "V", "Sc", "Sb", "Se", "Co",\
#"Cl", "Ca", "Ce", "Xe", "Cs", "Cr", "Cu", "La", "Li", "Tm", "Ti", "Te", "Tb", "Tc", "Yb", "Dy",\
#"I", "Y", "Ag", "Al", "As", "In"]

headers = ["H","Li","Be","B","C","N","O","F","Na","Mg","Al","Si","P","S","Cl",\
"K","Ca","Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn","Ga",\
"Ge","As","Se","Br","Rb","Sr","Y","Zr","Nb","Mo",\
"Tc","Ru","Rh","Pd","Ag","Cd","In","Sn",\
"Sb","Te","I","Xe","Cs","Ba","La","Ce","Pr",\
"Nd","Sm","Eu","Gd","Tb","Dy","Ho","Er",\
"Tm","Yb","Lu","Hf","Ta","W","Re","Os","Ir",\
"Pt","Au","Hg","Tl","Pb","Bi","Ac","Th","Pa",\
"U","Np","Am"] #hofmann

#headers = ["Ru", "Rb", "Rh", "Be", "Ba", "Bi", "Br", "H", "P", "Ge", "Gd", "Ga", "Pr", "Pu", "C",\
#"Pd", "Cd", "Ho", "Mg", "Mo", "Mn", "O", "S", "Eu", "Zr", "Er", "Ni",\
#"Na", "Nb", "Nd", "Fe", "B", "F", "Sr", "N", "Si", "Sn", "Sm", "V", "Sc", "Sb", "Se", "Co",\
#"Cl", "Ca", "Ce", "Xe", "Cs", "Cr", "Cu"]


unitformula = pd.read_csv("scaledformulae.csv", usecols=headers)
#unitformula = unitformula.drop("Unnamed: 0", axis=1)

### Setting models

In [5]:
from sklearn import linear_model, svm, kernel_ridge
from scipy.optimize import nnls 
from sklearn.metrics import mean_squared_error
estimators = [(linear_model.LinearRegression()),(linear_model.Ridge(alpha = 100)),(linear_model.ElasticNetCV())]

### Loop for RANSAC

In [ ]:
for base in estimators:
    alpha = 0.00
    #for step in range(1,3):
    ransacVols = linear_model.RANSACRegressor(base)
    ransacAlpha = linear_model.RANSACRegressor(base)
    
    ransacVols.fit(unitformula,vols.div(temps.Temperature*alpha+1, axis="index"))
    w = ransacVols.estimator_.coef_
    
    prvol = ransacVols.predict(unitformula) -1
    ransacAlpha.fit(temps, prvol)
    Talpha = ransacAlpha.estimator_.coef_
    Talpha = Talpha[0][0]
    
    print "\nUsing", base, "\n"
    print zip(unitformula.columns.values, np.mean(w, axis=0))
    print "\nAlphas: ", Talpha
    print "\nScore: ", ransacVols.score(unitformula, vols)

RANSAC is a regression method which is robust against outliers - to prevent the false negative and large results previously encountered.

This is a new loop; although most parts are the same a new modelling technique is being used, the loop allows for different base estimators to be used with RANSAC.

When each regressor is looped, after first iteration the results become extrememly errenuos
Using huber as a base estimator leads to memory error
ElasticNet causes memory error
ElasticNetCV also causing memory error